<a href="https://colab.research.google.com/github/mohsenahmadi2003/cnn_ai/blob/main/Session5_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **`Import`**

In [ ]:
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import time
import os
import copy
from torchvision import datasets, models, transforms
from __future__ import print_function
import torchsummary
import torchvision.models as models

# **`Initialization`**

In [ ]:
batch_size =256
num_class =10
num_epochs = 25
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# **`Dataset`**

In [ ]:
train_dataset = torchvision.datasets.CIFAR10("./cifar_train", train=True, transform=torchvision.transforms.ToTensor(), download=True)

test_dataset = torchvision.datasets.CIFAR10("./cifar_test", train=False, transform=torchvision.transforms.ToTensor(), download=True)


# **`Data Loader`**

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
one_train_batch_imgs, one_train_batch_lbls = next(iter(train_loader))
print(one_train_batch_imgs.shape)
print(one_train_batch_lbls)

In [ ]:
for index, (images, labels) in enumerate(train_loader):
    print(index, images.shape)

In [ ]:
dataloaders ={'train':train_loader,'test':test_loader}

# **`Model`**

---
in_features:

تعداد ورودی های لایه فولی کانکتد

out_features:

تعداد خروجی های لایه فولی کانکتد

---




In [ ]:
base = models.resnet18(pretrained=True).to(device)

layers = list(base.children())

l1 = nn.Sequential(*layers[:-5])
# print(l1)

l2 = nn.Sequential(*layers[5])
# print(l2)

l3 = nn.Sequential(*layers[6])
# print(l3)

l4 = nn.Sequential(*layers[7:9])
# print(l4)

l5 = nn.Sequential(layers[9])
# print(l5)

In [ ]:
def vis(imgs, idx):
    img = imgs[idx,...].abs().sum(1).squeeze(0)
    img = np.moveaxis(img.cpu().detach().numpy(), 0, -1)
    plt.imshow(img, interpolation='bilinear')
    # plt.Show()

In [ ]:
imgs, lbls = next(iter(dataloaders['train']))
imgs = imgs.to(device)
lbls = lbls.to(device)

idx = torch.randint(0, batch_size, size=(1,))

y1 = l1(imgs)
print(y1.shape)
# vis(y1, idx)

y2 = l2(y1)
print(y2.shape)

y3 = l3(y2)
print(y3.shape)

y4 = l4(y3)
print(y4.shape)

n1 = nn.Conv2d(in_channels=64, out_channels=512, kernel_size=1, stride=8).to(device)
z1 = n1(y1) * y4
z1 = z1.view(z1.size(0), -1)
print('z1: ', z1.shape)

n2 = nn.Conv2d(in_channels=128, out_channels=512, kernel_size=1, stride=4).to(device)
z2 = n2(y2) * y4
z2 = z2.view(z2.size(0), -1)
print('z2: ', z2.shape)

n3 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=1, stride=2).to(device)
z3 = n3(y3) * y4
z3 = z3.view(z3.size(0), -1)
print('z3: ', z3.shape)

zt = torch.cat((z1, z2, z3), dim=1)
print(zt.shape)

fc = nn.Linear(in_features=512*3, out_features=10, bias=False).to(device)
output = fc(zt)
output.shape

In [ ]:

class convnet(nn.Module):
    def __init__(self, num_class):
        super(convnet, self).__init__()
        self.base = models.resnet18(pretrained=True)
        self.base.fc = nn.Linear(in_features=512, out_features=10, bias=False)


    def forward(self, x):
        y = self.base(x)
        return y

In [ ]:
model = convnet(num_class).to(device)
print(model)

In [ ]:
flag = False
for name, param in model.named_parameters():
    if not 'layer3.0' in name or flag:
        param.requires_grad = True
        print(name)
        flag = True
    else:
        param.requires_grad = False
    

In [ ]:
imgs, lbls = next(iter(dataloaders['train']))
imgs = imgs.to(device)
lbls = lbls.to(device)
print(imgs.shape)

import numpy as np
import matplotlib.pyplot as plt

idx = torch.randint(0, batch_size, size=(1,))
img = imgs[idx,...].squeeze(0)

img = np.moveaxis(img.cpu().numpy(), 0, -1)
img.shape

plt.imshow(img, interpolation='bilinear')

# **Config**

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# **Train**

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'test':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.001,momentum=0.9)

In [ ]:
model, hist = train_model(model, dataloaders, criterion, optimizer, num_epochs)